In [1]:
import os
import re
import torch
from torch import nn
from tqdm import tqdm
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    DataCollatorWithPadding,
    BitsAndBytesConfig,
)
from tokenizers.processors import TemplateProcessing
from trl import SFTConfig, SFTTrainer
from peft import (
    prepare_model_for_kbit_training, 
    LoraConfig, 
    get_peft_model,
    TaskType,
)
from datasets import load_dataset
from accelerate import Accelerator

/opt/conda/envs/qa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 데이터셋 정의

In [6]:
repo = 'KorQuAD/squad_kor_v2'
tokenizer = AutoTokenizer.from_pretrained(repo)
dataset = load_dataset("csv", data_files="/home/jovyan/work/prj_data/open/train.csv")
max_length = 1280

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.backend_tokenizer.post_processor = TemplateProcessing(
    single="<s> $A </s>",
    pair="<s> $A <s> $B </s>",
    special_tokens=[
        ("<s>", tokenizer.convert_tokens_to_ids("<s>")),
        ("</s>", tokenizer.convert_tokens_to_ids("</s>"))
    ],
)

def preprocess_function(examples):
    question, context, answer = examples["question"], examples["context"], examples["answer"]
    def preprocess_text(text):
        text = text.replace('\n', ' ')
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    question = preprocess_text(question)
    context = preprocess_text(context)
    answer = preprocess_text(answer)
    
    inputs = tokenizer(
        question,
        context,
        return_offsets_mapping=True,
        truncation=False, 
        # truncation=True,
        # max_length=max_length, 
        # padding="max_length",
    )
    
    start_char = context.find(answer)
    end_char = start_char + len(answer)

    # offset_mapping: [(token1 start, token1 end), (token2 ~, ), ...]
    offset= inputs.pop("offset_mapping")
    
    # sequence_ids: (token=None, question=0, context=1)
    sequence_ids = inputs.sequence_ids(0)

    # 컨텍스트의 시작 및 마지막을 찾는다.
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # 만일 정답이 컨텍스트에 완전히 포함되지 않는다면, 레이블은 (0, 0)임
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_position = 0
        end_position = 0
    else:
        # 그렇지 않으면 정답의 시작 및 마지막 인덱스
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_position = idx - 1

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_position = idx + 1

    inputs["start_positions"] = start_position
    inputs["end_positions"] = end_position
    return inputs

# 데이터 프레임을 전처리합니다
preprocess_function(dataset["train"][0])
# train_dataset = dataset["train"].map(preprocess_function)
train_dataset = train_dataset.remove_columns(['id', 'context', 'question', 'answer'])

Map:   3%|▎         | 999/33716 [00:01<00:53, 614.59 examples/s]


TypeError: 'NoneType' object is not subscriptable

In [11]:
test_idx = 15
print(tokenizer.decode(train_dataset[test_idx]["input_ids"][train_dataset[test_idx]["start_positions"] : train_dataset[test_idx]["end_positions"] + 1]))

NameError: name 'train_dataset' is not defined

# 모델 정의

In [5]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
print("start")
model = AutoModelForQuestionAnswering.from_pretrained(
        repo,
        quantization_config=quantization_config,
        device_map={"":0},
        torch_dtype="auto",
)
print("end")

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=['up_proj', 
                    'down_proj', 
                    'gate_proj', 
                    'k_proj', 
                    'q_proj', 
                    'v_proj', 
                    'o_proj'],
    task_type="QUESTION_ANSWERING"
)

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

accelerater = Accelerator()
model, tokenizer = accelerater.prepare(model, tokenizer)
model.config.use_cache = True

start


Loading checkpoint shards: 100%|██████████| 9/9 [00:11<00:00,  1.22s/it]
Some weights of LlamaForQuestionAnswering were not initialized from the model checkpoint at charlieCs/Open-Solar-ko-10B-dacon-qa and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


end


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


# Loss 정의

# 학습

In [7]:
import wandb
wandb.login()
os.environ["TOKENIZERS_PARALLELISM"] = "false"

training_args = TrainingArguments(
    output_dir="test",
    num_train_epochs=1,
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True
    save_steps=0.1,
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: uijinkim. Use `wandb login --relogin` to force relogin
/opt/conda/envs/qa/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [8]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


/opt/conda/envs/qa/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

# Inference

In [2]:
import torch
import pandas as pd
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from tqdm import tqdm

CHECK_POINT = "/home/jovyan/work/ai_chat_qa_task/code/huggingface/SOLAR_QA/checkpoint-4215"
TEST_fOLDER = '/home/jovyan/work/prj_data/open/test.csv'
OUTPUT = "test"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
csv = pd.read_csv(TEST_fOLDER)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# 모델 및 토크나이저 로드
config = PeftConfig.from_pretrained(CHECK_POINT)
model = AutoModelForQuestionAnswering.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(model, CHECK_POINT)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

ValueError: Can't find 'adapter_config.json' at 'KorQuAD/squad_kor_v2'

In [5]:
TEST_fOLDER = '/home/jovyan/work/prj_data/open/train.csv'
csv = pd.read_csv(TEST_fOLDER)
idx = 4

def get_prediction(question, context):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=False)
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)

    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits)
    print(answer_start, answer_end)
    
    answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end+1])
    return answer

predictions = []
i = 0
for _, row in tqdm(csv.iterrows(), total=len(csv)):
    if i == idx:
        answer = get_prediction(row['question'], row['context'])
        predictions.append(answer)
        print(row['question'])
        print("---")
        print(row['context'])
        print("---")
        print(answer)
        break
    i+=1

  0%|          | 4/33716 [00:00<1:02:25,  9.00it/s]

tensor(7, device='cuda:0') tensor(1, device='cuda:0')
PM9A3 E1.S의 연속쓰기 속도는
---
 삼성전자가 OCP(오픈 컴퓨트 프로젝트)의 규격을 만족하는 데이터센터 전용 고성능 SSD ‘PM9A3 E1.S’를 양산한다고 24일 밝혔다. 

 

 OCP는 글로벌 데이터센터 관련 기업들이 효율적인 데이터센터 개발과 운영에 필요한 하드웨어와 소프트웨어의 표준을 정립하는 기구다. 

 

 이번 제품은 업계최초 6세대 V낸드를 기반으로 한 데이터센터 전용 SSD로, OCP의 NVMe Cloud SSD 표준을 지원하며, 데이터센터에서 요구하는 성능, 전력 효율, 보안 등을 각각 최고 수준의 솔루션으로 제공한다. 

 

 특히 전력 효율이 업계 최고 수준으로 높아 데이터센터 운영 비용을 절감할 수 있으며, 최근 화두가 되고 있는 탄소 저감 효과도 기대할 수 있다. 

 

 PM9A3 E1.S의 전력 효율은 연속쓰기 성능을 기준으로 할 때 1와트(W)당 283MB/s를 지원하며, 이는 이전 세대인 5세대 V낸드 기반 PM983a M.2 보다 약 50% 향상됐다. 

 

 지난해 전세계 서버용으로 출하된 HDD(하드 디스크 드라이브)를 모두 PM9A3 E1.S 4TB로 대체하면 1년간 절감할 수 있는 전력량이 1484GWh에 이른다. 

 

 PM9A3 E1.S의 연속쓰기 속도는 3000MB/s로 이전 세대인 제품 대비 연속 쓰기 속도가 약 배 증가했으며, 임의읽기 속도(750K IOPS)와 임의쓰기 속도(160K IOPS)도 각각 40%, 150% 향상됐다. 

 

 이번 제품은 사용자 데이터 암호화와 같은 기본적인 보안 기능 뿐만 아니라 안티롤백, 보안 부팅 등 다양한 보안 솔루션을 제공한다. 

 

 안티롤백은 보안이 취약한 하위 버전의 펌웨어가 다운로드 되지 못하도록 막는 기능으로, PM9A3 E1.S는 보안 취약점이 발견된 펌웨어에 대해서는 이력을 따로 저장해놓고 해당 버전을 다운로드할 경우 정상적으로 처